# Домашнее задание № 3. Исправление опечаток

## 1. Доп. ранжирование по вероятности (3 балла)

Дополните get_closest_hybrid_match в семинаре так, чтобы из кандадатов с одинаковым расстоянием редактирования выбиралось наиболее вероятное.

In [3]:
!pip install textdistance

In [1]:
!wget https://github.com/mannefedov/compling_nlp_hse_course/raw/refs/heads/master/notebooks/spelling/data.zip

--2025-06-29 21:41:30--  https://github.com/mannefedov/compling_nlp_hse_course/raw/refs/heads/master/notebooks/spelling/data.zip
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/refs/heads/master/notebooks/spelling/data.zip [following]
--2025-06-29 21:41:31--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/refs/heads/master/notebooks/spelling/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18787000 (18M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]  17.92M  5.59MB/s    in 3.4s    

2025-06

In [3]:
!unzip data.zip

Archive:  data.zip
  inflating: correct_sents.txt       
  inflating: sents_with_mistakes.txt  
  inflating: wiki_data.txt           
  inflating: __MACOSX/._wiki_data.txt  


In [13]:
import re
from string import punctuation
from collections import Counter, defaultdict
import numpy as np
from tqdm import tqdm
import textdistance
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_distances
import random

In [14]:
BAD_PATH = './sents_with_mistakes.txt'
TRUE_PATH = './correct_sents.txt'
WIKI_PATH = './wiki_data.txt'

In [15]:
bad_sents = open(BAD_PATH, encoding='utf8').read().splitlines()
true_sents = open(TRUE_PATH, encoding='utf8').read().splitlines()

In [16]:
PUNCT_ALL = punctuation + "«»—…“”"

def align_words(sent1: str, sent2: str):
    t1 = [tok.strip(PUNCT_ALL) for tok in sent1.lower().split() if tok.strip(PUNCT_ALL)]
    t2 = [tok.strip(PUNCT_ALL) for tok in sent2.lower().split() if tok.strip(PUNCT_ALL)]
    return list(zip(t1, t2))

mistake_pairs = []
for tr, bd in zip(true_sents, bad_sents):
    for w_true, w_bad in align_words(tr, bd):
        mistake_pairs.append((w_true, w_bad))

SAMPLE_SIZE = 5000
random.seed(42)
eval_pairs = random.sample(mistake_pairs, min(SAMPLE_SIZE, len(mistake_pairs)))

wiki_text = open(WIKI_PATH, encoding='utf8').read().lower()
vocab = Counter(re.findall(r"\w+", wiki_text))
N = sum(vocab.values())

def P(word: str) -> float:
    return vocab[word] / N if word in vocab else 0.0

def is_mistaken(word: str) -> bool:
    return word not in vocab

In [17]:
word_list = list(vocab.keys())
vectorizer = CountVectorizer(analyzer='char', ngram_range=(1,3), max_features=10000)
X = vectorizer.fit_transform(word_list)
id2word = {i: word_list[i] for i in range(len(word_list))}

In [18]:
def get_closest_vec(text: str, topn: int = 20):
    v = vectorizer.transform([text])
    dists = cosine_distances(v, X)[0]
    idx = np.argsort(dists)[:topn]
    return [(id2word[i], dists[i]) for i in idx]

def get_closest_hybrid_match(text: str, topn: int = 3, max_vec: int = None):
    if max_vec is None:
        max_vec = topn * 4
    vec_cands = get_closest_vec(text, topn=max_vec)
    scored = []
    for w, _ in vec_cands:
        dist = textdistance.damerau_levenshtein.distance(text, w)
        prob = P(w)
        scored.append((w, dist, prob))
    scored.sort(key=lambda x: (x[1], -x[2]))
    return [(w, d) for w, d, p in scored[:topn]]

In [20]:
total = 0
correct = 0
mistakes_total = 0
mistakes_fixed = 0
correct_total = 0
correct_broken = 0

for w_true, w_bad in tqdm(eval_pairs):
    if is_mistaken(w_bad):
        pred = get_closest_hybrid_match(w_bad, topn=1)[0][0]
    else:
        pred = w_bad
    total += 1
    if pred == w_true:
        correct += 1
    if w_true != w_bad:
        mistakes_total += 1
        if pred == w_true:
            mistakes_fixed += 1
    else:
        correct_total += 1
        if pred != w_true:
            correct_broken += 1

100%|███████████████████████████████████████| 5000/5000 [03:30<00:00, 23.78it/s]


In [21]:
print("Гибридный метод с вероятностью")
print(f"Точность: {correct/total:.4f}")
print(f"Процент исправленных ошибок: {mistakes_fixed/mistakes_total:.4f}")
print(f"Процент испорченных корректных слов: {correct_broken/correct_total:.4f}\n")

Гибридный метод с вероятностью
Точность: 0.8568
Процент исправленных ошибок: 0.4554
Процент испорченных корректных слов: 0.0844



## 2.  Symspell (7 баллов)

Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Он основан только на одной операции - удалении символа. Описание алгоритма по шагам:

1) Составляется словарь правильных слов  
2) На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово  (!) 
3) Для выбора исправления для слова с опечаткой генерируются все варианты удаления, из них выбираются те, что есть в словаре удалений, построенного на шаге 2. Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления  
4) Если в словаре удалений есть несколько вариантов, то выбирается удаление, которому соответствует наиболее вероятное правильное слово  


Оцените качество полученного алгоритма теми же тремя метриками.

In [22]:
def generate_deletes(word: str, max_edit: int = 2):
    deletes = set()
    queue = [word]
    for d in range(max_edit):
        next_queue = []
        for w in queue:
            for i in range(len(w)):
                var = w[:i] + w[i+1:]
                if var not in deletes:
                    deletes.add(var)
                    next_queue.append(var)
        queue = next_queue
    return deletes


In [23]:
MAX_EDIT = 2
delete_dict = defaultdict(set)
for w in vocab:
    for d in generate_deletes(w, MAX_EDIT):
        delete_dict[d].add(w)


In [24]:
def symspell_correct(word: str, max_edit: int = MAX_EDIT) -> str:
    if word in vocab:
        return word
    cands = set()
    for d in generate_deletes(word, max_edit):
        if d in delete_dict:
            cands.update(delete_dict[d])
    if not cands:
        return word
    # фильтруем по реальному расстоянию
    scored = []
    for w in cands:
        dist = textdistance.damerau_levenshtein.distance(word, w)
        if dist <= max_edit:
            scored.append((w, dist, P(w)))
    if not scored:
        return word
    scored.sort(key=lambda x: (x[1], -x[2]))
    return scored[0][0]

In [25]:
total = 0
correct = 0
mistakes_total = 0
mistakes_fixed = 0
correct_total = 0
correct_broken = 0

for w_true, w_bad in tqdm(eval_pairs):
    if is_mistaken(w_bad):
        pred = symspell_correct(w_bad)
    else:
        pred = w_bad
    total += 1
    if pred == w_true:
        correct += 1
    if w_true != w_bad:
        mistakes_total += 1
        if pred == w_true:
            mistakes_fixed += 1
    else:
        correct_total += 1
        if pred != w_true:
            correct_broken += 1

print("SymSpell")
print(f"Точность: {correct/total:.4f}")
print(f"Процент исправленных ошибок: {mistakes_fixed/mistakes_total:.4f}")
print(f"Процент испорченных корректных слов: {correct_broken/correct_total:.4f}")

100%|████████████████████████████████████| 5000/5000 [00:00<00:00, 15515.24it/s]

SymSpell
Точность: 0.8726
Процент исправленных ошибок: 0.4726
Процент испорченных корректных слов: 0.0688
